In [115]:
# Стандартизация
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('data/occupancy_even.csv')

scaler = preprocessing.StandardScaler()

names = df.columns
scaled_data = scaler.fit_transform(df)

df_std = pd.DataFrame(scaled_data, columns=names)


In [116]:
# Создание и обучение классификатора
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

x = df_std.iloc[:, :-1].values  # все колонки до последней - переменные функции
y = df.iloc[:, -1].values  # в качестве результирующего значения берем последнюю колонку

# делим набор данных на обучающий и тестовый
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(x_train, y_train)  # идентифицирует и изучает параметры модели из обучающего набора данных


KNeighborsClassifier(n_neighbors=10)

In [117]:
# Оценка качества классификатора
y_pred = classifier.predict(x_test)  # прогноз на основе тестовых данных

# классификационная оценка точности
print("Точность:\n", accuracy_score(y_test, y_pred), "\n")

# вычисляет матрицу ошибок для оценки точности классификации
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred), "\n")


Точность:
 0.9821073558648111 

Матрица ошибок:
 [[269   8]
 [  1 225]] 



In [118]:
# Генератор входных данных
import pandas as pd
import numpy as np

n = 15  # количество входных объектов

# генерируем параметры в диапазоне, большем допустимого интервала
temp = np.random.uniform(low=15.0, high=30.0, size=(n,))
humid = np.random.uniform(low=10.0, high=40.0, size=(n,))
light = np.random.uniform(low=0.0, high=1200.0, size=(n,))
co2 = np.random.uniform(low=300.0, high=2000.0, size=(n,))
humid_ratio = np.random.uniform(low=0.001, high=0.007, size=(n,))
occupancy = np.zeros((n,))

matrix_aux = np.vstack([temp, humid, light, co2, humid_ratio, occupancy])
matrix = np.transpose(matrix_aux)
df_input = pd.DataFrame(matrix, columns=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'])

print(df_input)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
0     18.136202  31.858187  457.202975  1296.095805       0.004108        0.0
1     24.165500  29.480464  151.836255   848.832122       0.005877        0.0
2     17.060492  19.970922  542.209615   533.254476       0.003217        0.0
3     25.506268  21.017839  812.901865  1662.483131       0.004609        0.0
4     15.824123  16.508730  695.450783   633.355417       0.001177        0.0
5     17.910163  28.136127  438.945401  1393.928745       0.006219        0.0
6     29.013186  20.784744  659.301007   882.637374       0.003562        0.0
7     23.299334  30.895699   11.024128  1471.714233       0.004622        0.0
8     28.615650  17.748791  740.269871  1600.068199       0.006959        0.0
9     23.172194  28.939258  567.277554  1102.303801       0.002324        0.0
10    23.672578  21.877401  144.451501  1628.179072       0.003566        0.0
11    20.070492  27.286837  704.326867   669.812453       0.0033

In [119]:
# Стандартизируем тестовые данные
import pandas as pd

input_scaled = scaler.transform(df_input)

df_input_std = pd.DataFrame(input_scaled, columns=names)
print(df_input_std)


    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
0     -3.219282  2.448686  0.817953  1.711617      -0.039385  -0.963873
1      2.361135  1.499881 -0.379489  0.245272       2.749593  -0.963873
2     -4.214905 -2.294798  1.151292 -0.789343      -1.444308  -0.963873
3      3.602083 -1.877038  2.212765  2.912810       0.750024  -0.963873
4     -5.359227 -3.676349  1.752201 -0.461164      -4.660318  -0.963873
5     -3.428492  0.963438  0.746359  2.032360       3.288324  -0.963873
6      6.847910 -1.970052  1.610446  0.356102      -0.900367  -0.963873
7      1.559455  2.064615 -0.931659  2.287378       0.770043  -0.963873
8      6.479971 -3.181516  1.927951  2.708184       4.455042  -0.963873
9      1.441781  1.283919  1.249592  1.076274      -2.851896  -0.963873
10     1.904911 -1.534039 -0.408447  2.800345      -0.893199  -0.963873
11    -1.429000  0.624538  1.787007 -0.341640      -1.216595  -0.963873
12     7.324308 -0.985419 -0.780698  0.482614      -3.478692  -0

In [120]:
# Удаляем значения вне доверительного интервала
std = df_std.std()

for_deletion = df_input_std.index[
    (df_input_std.Temperature < (-3 * std.Temperature)) |
    (df_input_std.Temperature > (3 * std.Temperature)) |
    (df_input_std.Humidity < (-3 * std.Humidity)) |
    (df_input_std.Humidity > (3 * std.Humidity)) |
    (df_input_std.Light < (-3 * std.Light)) |
    (df_input_std.Light > (3 * std.Light)) |
    (df_input_std.CO2 < (-3 * std.CO2)) |
    (df_input_std.CO2 > (3 * std.CO2)) |
    (df_input_std.HumidityRatio < (-3 * std.HumidityRatio)) |
    (df_input_std.HumidityRatio > (3 * std.HumidityRatio))
    ].tolist()

df_input_clean = df_input.drop(for_deletion)
df_input_clean_std = df_input_std.drop(for_deletion)

print(df_input_clean, '\n')
print(df_input_clean_std)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
1     24.165500  29.480464  151.836255   848.832122       0.005877        0.0
7     23.299334  30.895699   11.024128  1471.714233       0.004622        0.0
9     23.172194  28.939258  567.277554  1102.303801       0.002324        0.0
10    23.672578  21.877401  144.451501  1628.179072       0.003566        0.0
11    20.070492  27.286837  704.326867   669.812453       0.003361        0.0 

    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
1      2.361135  1.499881 -0.379489  0.245272       2.749593  -0.963873
7      1.559455  2.064615 -0.931659  2.287378       0.770043  -0.963873
9      1.441781  1.283919  1.249592  1.076274      -2.851896  -0.963873
10     1.904911 -1.534039 -0.408447  2.800345      -0.893199  -0.963873
11    -1.429000  0.624538  1.787007 -0.341640      -1.216595  -0.963873


In [121]:
# Подаем полученные данные на вход классификатору

input_x = df_input_clean_std.iloc[:, :-1].values

input_y = classifier.predict(input_x)
probs = classifier.predict_proba(input_x)
probs = np.transpose(probs)

df_input_clean.Occupancy = input_y
df_input_clean['Probability0'] = probs[0]
df_input_clean['Probability1'] = probs[1]

print(df_input_clean)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy  \
1     24.165500  29.480464  151.836255   848.832122       0.005877          1   
7     23.299334  30.895699   11.024128  1471.714233       0.004622          0   
9     23.172194  28.939258  567.277554  1102.303801       0.002324          1   
10    23.672578  21.877401  144.451501  1628.179072       0.003566          0   
11    20.070492  27.286837  704.326867   669.812453       0.003361          1   

    Probability0  Probability1  
1            0.0           1.0  
7            1.0           0.0  
9            0.1           0.9  
10           0.9           0.1  
11           0.0           1.0  
